In [1]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
from pathlib import Path
import netCDF4 as netcdf
from shapely.geometry import Point
from concorde.tools import readFort22

import xarray as xr
from tqdm import tqdm as tqdm

In [2]:
pathin = Path(r'/media/tacuevas/Extreme SSD/batch02')
runs = sorted([x for x in os.listdir(pathin) if x.isdigit()])

In [3]:
for run in tqdm(runs):
    # with netcdf.Dataset(pathin/run/'maxele.63.nc', 'r') as melev:
    melev = netcdf.Dataset(pathin/run/'maxele.63.nc', 'r')
    f63 = netcdf.Dataset(pathin/run/'fort.63.nc')
    f22 = readFort22(pathin/run/'fort.22')
    startStorm = f22[f22['wind_speed'] > 1.03].index[0]
    
    t0 = pd.to_datetime(melev['time'].units.split('since ')[1]).tz_localize(None)
    timeMaxElevSeconds = pd.to_timedelta(melev['time_of_zeta_max'][:].data, unit = 's')
    timeMaxElev = t0 + timeMaxElevSeconds
    timeBeforeStart = timeMaxElev < startStorm ## False is "correct"

    datesF63 = [t0 + pd.Timedelta(seconds = x) for x in f63['time'][:]]
    f63IndexStartStorm = datesF63.index(startStorm)
    maxF63 = f63['zeta'][f63IndexStartStorm:, :].data.max(axis = 0)
    timeMaxF63 = f63['zeta'][f63IndexStartStorm:, :].data.argmax(axis = 0)
    maxMelev = melev['zeta_max'][:].data
    
    maxMelevCorr = maxMelev.copy()
    maxMelevCorr[timeBeforeStart] = maxF63[timeBeforeStart]
    timeMaxMelevCorr = melev['time_of_zeta_max'][:].data.copy()
    timeMaxMelevCorr[timeBeforeStart] = timeMaxF63[timeBeforeStart]


    nnodes = range(melev['zeta_max'].shape[0])
    dims = ['time', 'node']
    coords = {'time': [0], 'node': nnodes}
    
    dummy1 = maxMelevCorr.reshape((1, len(maxMelevCorr)))
    dummy2 = np.ma.masked_invalid(dummy1)
    dummy3 = dummy2.filled(fill_value = -99999.0)

    dummy4 = timeMaxMelevCorr.reshape((1, len(timeMaxMelevCorr)))
    dummy5 = np.ma.masked_invalid(dummy4)
    dummy6 = dummy5.filled(fill_value = -99999.0)

    ds = xr.Dataset({
                    'zeta_max':xr.DataArray(data = melev['zeta_max'],
                                        dims = ['time', 'node'],
                                        coords = coords),
                    'time_of_zeta_max':xr.DataArray(data = melev['time_of_zeta_max'],
                                        dims = ['time', 'node'],
                                        coords = coords),
                    'zeta_max_corr':xr.DataArray(data = dummy3,
                                          dims = ['time', 'node'],
                                          coords = coords),
                    'time_of_zeta_max_corr':xr.DataArray(data = dummy6,
                                        dims = ['time', 'node'],
                                        coords = coords),
                    'corrected':xr.DataArray(data = (timeBeforeStart*1).reshape((1, len(timeBeforeStart))),
                                        dims = ['time', 'node'],
                                        coords = coords),
                    'depth':xr.DataArray(data = melev['depth'],
                                        dims = ['time', 'node'],
                                        coords = coords),
                    'element': xr.DataArray(data = melev['element'][:, :].data,
                                            dims = ['nele', 'nvertex']),
                    'x': xr.DataArray(data = melev['x'][:].data,
                                        dims = ['node'],
                                        coords = {'node': nnodes}),
                    'y': xr.DataArray(data = melev['y'][:].data,
                                        dims = ['node'],
                                        coords = {'node': nnodes})
                    })

    ds['zeta_max_corr'].attrs['long_name'] = melev['zeta_max'].long_name
    ds['zeta_max_corr'].attrs['units'] = melev['zeta_max'].units
    
    ds.to_netcdf(pathin/run/'maxele63Corr.nc')



    # gdf = gpd.GeoDataFrame(geometry = [Point(x, y) for x, y in zip(melev['x'][:].data, melev['y'][:].data)])
    # gdf['max_melev'] = maxMelev
    # gdf['max_f63'] = maxF63
    # gdf['before_start'] = timeBeforeStart
    # gdf['max_melev_corr'] = maxMelevCorr
    # gdf['diff'] = maxMelevCorr - maxMelev
    # gdf.to_file(r'/mnt/drive1/Insyncs/NCSU/thesis/gis/gpkg/correctedTimePeakWL0000.gpkg', driver = 'GPKG')

100%|██████████| 1813/1813 [11:53:37<00:00, 23.62s/it]  
